# [Julia](https://julialang.org/) Notebook for the Discussion on Aperiodic Sampling

Survey: L. Hetel et al., "Recent developments on the stability of systems with aperiodic sampling: An overview", Automatica, 2017.

In [1]:
#Pkg.add("ControlSystems")
#Pkg.add("SCS")

In [2]:
using ControlSystems

In [3]:
using Plots
#plotlyjs()
#pyplot()

In [4]:
# Helper function to generate the iterates Aᵏx₀, for k ≧ 0
function generateDTtraj(A, x0, nbSteps)
    traj = zeros(length(x0), nbSteps+1)
    traj[:,1] = x0
    for k=1:nbSteps
        traj[:,k+1] = A*traj[:,k]
    end
    return traj
end

generateDTtraj (generic function with 1 method)

## Issues with time-varying sampling time

In [5]:
# CT ss data
A1 = [1 3; 2 1]
B1 = [1; 0.6]
C1 = zeros(1, 2)
D1 = zeros(1)
K1 = -[1 6]
# Sampling periods
h1 = 0.18
h2 = 0.54;

In [6]:
sysc = ss(A1, B1, C1, D1);

In [7]:
real(eigvals(A1+B1*K1))  # make sure the CT system is stable

2-element Array{Float64,1}:
 -1.3
 -1.3

In [8]:
?c2d  # same function as in Matlab

search: c2d c2d_poly2poly c2d_roots2poly feedback2dof chr2ind Complex32



`[sysd, x0map] = c2d(sys, Ts, method=:zoh)`

Convert the continuous system `sys` into a discrete system with sample time `Ts`, using the provided method. Currently only `:zoh` and `:foh` are provided.

Returns the discrete system `sysd`, and a matrix `x0map` that transforms the initial conditions to the discrete domain by `x0_discrete = x0map*[x0; u0]`


In [9]:
sysd1 = c2d(sysc, h1);  # discrete-time system for period h1

In [10]:
# Let's make sure this discretization is correct
println(expm(h1*[A1 B1; 0 0 0]))  # see Lemma 5.4.2
println(sysd1[1].A)
println(sysd1[1].B)

[1.31548 0.667648 0.237419; 0.445099 1.31548 0.159551; 0.0 0.0 1.0]
[1.31548 0.667648; 0.445099 1.31548]
[0.237419; 0.159551]


In [11]:
Λ₁ = sysd1[1].A + sysd1[1].B * K1  # closed loop DT system for period h1

2×2 Array{Float64,2}:
 1.07807   -0.756866
 0.285548   0.358176

In [12]:
abs.(eigvals(Λ₁))

2-element Array{Float64,1}:
 0.776053
 0.776053

In [13]:
sysd2 = c2d(sysc, h2);  # discrete-time system for period h2
Λ₂ = sysd2[1].A + sysd2[1].B * K1 
abs.(eigvals(Λ₂))

2-element Array{Float64,1}:
 0.708291
 0.708291

So both periodic sampled-data systems are stable in DT. What if the sampling period now switches between h1 and h2 every time. Let's look over two periods to recover time-invariance.

In [14]:
Λₐ = Λ₁ * Λ₂

2×2 Array{Float64,2}:
 1.15571  -2.94245
 1.06942  -2.46131

In [15]:
abs.(eigvals(Λₐ))

2-element Array{Float64,1}:
 0.300651
 1.00495 

We lost stability!

In [16]:
traj = generateDTtraj(Λₐ, [0.1, 0.1], 100)
plot(traj[1,:], line = :step)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 20 
 
 
 40 
 
 
 60 
 
 
 80 
 
 
 100 
 
 
 -0.3 
 
 
 -0.2 
 
 
 -0.1 
 
 
 0.0 
 
 
 0.1 
 
 
 0.2 
 
 
 0.3 
 
<polyline clip-path="url(#clip02)" style="stroke:#0099ff; stroke-width:0.8; stroke-opacity:1; fill:none" points="
 39.1436,141.474 44.5553,141.474 44.5553,288.579 49.967,288.579 49.967,87.0698 55.3787,87.0698 55.3787,305.713 60.7904,305.713 60.7904,81.1363 66.2022,81.1363 
 66.2022,308.283 71.6139,308.283 71.6139,79.574 77.0256,79.574 77.0256,309.547 82.4373,309.547 82.4373,78.3966 87.849,78.3966 87.849,310.702 93.2607,310.702 
 93.2607,77.2437 98.6724,77.2437 98.6724,311.858 104.084,311.858 104.084,76.0827 109.496,76.0827 109.496,313.025 114.908,313.025 114.908,74.9105 120.319,74.9105 
 120.319,314.203 125.731,314.203 125.731,73.7267 131.143,73.7267 131.143,315.392 136.554,315.392 136.554,72.5311 141.966,72.5311 141.966,316.594 147.378,316.594 
 147.378,71.3237 152.79,71.3237 152.79,317.807 158.201,317.807 158.201,70.1043 163.613,70.1043 163.613,319.033 169.025,319.033 169.025,68.8728 174.436,68.8728 
 174.436,320.27 179.848,320.27 179.848,67.6291 185.26,67.6291 185.26,321.52 190.672,321.52 190.672,66.373 196.083,66.373 196.083,322.782 201.495,322.782 
 201.495,65.1045 206.907,65.1045 206.907,324.057 212.318,324.057 212.318,63.8234 217.73,63.8234 217.73,325.345 223.142,325.345 223.142,62.5296 228.554,62.5296 
 228.554,326.645 233.965,326.645 233.965,61.223 239.377,61.223 239.377,327.958 244.789,327.958 244.789,59.9034 250.2,59.9034 250.2,329.284 255.612,329.284 
 255.612,58.5707 261.024,58.5707 261.024,330.623 266.436,330.623 266.436,57.2248 271.847,57.2248 271.847,331.976 277.259,331.976 277.259,55.8656 282.671,55.8656 
 282.671,333.342 288.082,333.342 288.082,54.4928 293.494,54.4928 293.494,334.721 298.906,334.721 298.906,53.1065 304.318,53.1065 304.318,336.114 309.729,336.114 
 309.729,51.7064 315.141,51.7064 315.141,337.522 320.553,337.522 320.553,50.2924 325.964,50.2924 325.964,338.942 331.376,338.942 331.376,48.8644 336.788,48.8644 
 336.788,340.378 342.2,340.378 342.2,47.4222 347.611,47.4222 347.611,341.827 353.023,341.827 353.023,45.9657 358.435,45.9657 358.435,343.291 363.846,343.291 
 363.846,44.4948 369.258,44.4948 369.258,344.769 374.67,344.769 374.67,43.0093 380.082,43.0093 380.082,346.262 385.493,346.262 385.493,41.509 390.905,41.509 
 390.905,347.769 396.317,347.769 396.317,39.9939 401.728,39.9939 401.728,349.292 407.14,349.292 407.14,38.4637 412.552,38.4637 412.552,350.83 417.964,350.83 
 417.964,36.9184 423.375,36.9184 423.375,352.383 428.787,352.383 428.787,35.3577 434.199,35.3577 434.199,353.951 439.61,353.951 439.61,33.7816 445.022,33.7816 
 445.022,355.535 450.434,355.535 450.434,32.1898 455.846,32.1898 455.846,357.135 461.257,357.135 461.257,30.5822 466.669,30.5822 466.669,358.75 472.081,358.75 
 472.081,28.9587 477.492,28.9587 477.492,360.382 482.904,360.382 482.904,27.3191 488.316,27.3191 488.316,362.029 493.728,362.029 493.728,25.6632 499.139,25.6632 
 499.139,363.694 504.551,363.694 504.551,23.9909 509.963,23.9909 509.963,365.374 515.374,365.374 515.374,22.302 520.786,22.302 520.786,367.071 526.198,367.071 
 526.198,20.5964 531.61,20.5964 531.61,368.785 537.021,368.785 537.021,18.8738 542.433,18.8738 542.433,370.516 547.845,370.516 547.845,17.1342 553.256,17.1342 
 553.256,372.265 558.668,372.265 558.668,15.3773 564.08,15.3773 564.08,374.03 569.492,374.03 569.492,13.603 574.903,13.603 574.903,375.813 580.315,375.813 
 580.315,11.811 
 "/>
 
 
 
 
 y1

In [17]:
# A converse example
# CT ss data
A2 = [0 1; -2 0.1]
B2 = [0; 1]
C2 = zeros(1, 2)
D2 = zeros(1)
K2 = [1 0]
# Sampling periods
h3 = 2.126
h4 = 3.95;

Here, the CT system, and each periodically sampled system are unstable.

In [18]:
println(real(eigvals(A2+B2*K2)))
sysc2 = ss(A2, B2, C2, D2);
sysd3 = c2d(sysc2, h3);
sysd4 = c2d(sysc2, h4);
Λ₃ = sysd3[1].A + sysd3[1].B * K2
println(abs.(eigvals(Λ₃)))
Λ₄ = sysd4[1].A + sysd4[1].B * K2
println(abs.(eigvals(Λ₄)))

[0.05, 0.05]
[0.0647339, 1.08522]
[1.09271, 1.09271]


Now let's look again at a sequence of sampling periods switching between h3 and h4.

In [19]:
Λᵦ = Λ₃ * Λ₄
println(abs.(eigvals(Λᵦ)))

[0.102064, 0.821837]


With this particular sequence of sampling times, we get stability back!

In [20]:
traj = generateDTtraj(Λᵦ, [1, 1], 40)
plot(traj[1,:], line = :step)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 10 
 
 
 20 
 
 
 30 
 
 
 40 
 
 
 0.00 
 
 
 0.25 
 
 
 0.50 
 
 
 0.75 
 
 
 1.00 
 
<polyline clip-path="url(#clip02)" style="stroke:#0099ff; stroke-width:0.8; stroke-opacity:1; fill:none" points="
 38.8117,11.811 52.3493,11.811 52.3493,272.847 65.8869,272.847 65.8869,375.813 79.4245,375.813 79.4245,258.787 92.9621,258.787 92.9621,358.271 106.5,358.271 
 106.5,276.174 120.037,276.174 120.037,343.679 133.575,343.679 133.575,288.197 147.112,288.197 147.112,333.794 160.65,333.794 160.65,296.321 174.188,296.321 
 174.188,327.118 187.725,327.118 187.725,301.808 201.263,301.808 201.263,322.609 214.8,322.609 214.8,305.514 228.338,305.514 228.338,319.563 241.875,319.563 
 241.875,308.017 255.413,308.017 255.413,317.506 268.951,317.506 268.951,309.707 282.488,309.707 282.488,316.117 296.026,316.117 296.026,310.849 309.563,310.849 
 309.563,315.178 323.101,315.178 323.101,311.621 336.639,311.621 336.639,314.544 350.176,314.544 350.176,312.141 363.714,312.141 363.714,314.116 377.251,314.116 
 377.251,312.493 390.789,312.493 390.789,313.827 404.326,313.827 404.326,312.731 417.864,312.731 417.864,313.632 431.402,313.632 431.402,312.891 444.939,312.891 
 444.939,313.5 458.477,313.5 458.477,313 472.014,313 472.014,313.411 485.552,313.411 485.552,313.073 499.089,313.073 499.089,313.351 512.627,313.351 
 512.627,313.122 526.165,313.122 526.165,313.31 539.702,313.31 539.702,313.156 553.24,313.156 553.24,313.282 566.777,313.282 566.777,313.178 580.315,313.178 
 580.315,313.264 
 "/>
 
 
 
 
 y1

    This is related to well-known phenomena for switched systems. Switching can be destabilizing, as in the first example, or stabilizing, as in the second example. What happens is best understood in the phase plane for such 2-dimensional systems. See for example [this paper](http://liberzon.csl.illinois.edu/research/survey.pdf) of an introduction to switched systems.

## Maximum inter-sample time for aperiodic sampling

Reference: L. Mirkin, "Some Remarks on the Use of Time-Varying Delay to Model Sample-and-Hold Circuits", IEEE Transactions on Automatic Control, 2007,
and class discussion.

### SISO Plant

In [21]:
A = [0 1; 0 -0.1]
B = [0; 0.1]
C = zeros(1, 2)
D = zeros(1)
K = -[3.75 11.5]
sys = ss(A+B*K, B, K*(A+B*K), K*B);

In [22]:
#hM = pi/(2*norm(sys, Inf))
hM = pi/(2*norm(sys, Inf)[1])   # for older version of ControlSystems, Julia 0.5

1.365908483478383

### MIMO Plant and LMI

If the nominal system is MIMO, we can search for the maximum hM by dichotomy, using the scaled small gain theorem with matrix multipliers. This reduces to the feasibility of the following LMI by the bounded real lemma, see discussion in class. 

We pose the semi-definite programming using [JuMP](https://github.com/JuliaOpt/JuMP.jl), a modeling framework for optimization problems in Julia, we solve it using [SCS](https://github.com/JuliaOpt/SCS.jl) or [Mosek](https://github.com/JuliaOpt/Mosek.jl) as back-end SDP solver.

In [23]:
using JuMP
using SCS
#using Mosek  # Currently does not seem to work

As an exercise, let's redo the above computation, but less efficiently (see class discussion: this corresponds to not making the HS block commute with K on the "good side").

In [24]:
Acl = A+B*K; Bcl = B*K; Ccl = Acl; Dcl = Bcl;
n1 = size(Acl, 1); m1 = size(Dcl, 1);

In [25]:
hMtest = 1.366;
Γ = (4*hMtest^2/pi^2)^(-1);

In [26]:
optModel = Model(solver=SCSSolver());
#optModel = Model(solver=MosekSolver());

In [27]:
@variable(optModel, P[1:n1,1:n1], SDP);
@variable(optModel, Y[1:m1,1:m1], SDP);

In [28]:
myeps = 1  # By homogeneity, we can replace > 0 by >= I (since SDP solvers handle only >=)
@SDconstraint(optModel, P >= myeps*eye(n1));
@SDconstraint(optModel, Y >= myeps*eye(m1));

In [29]:
L11 = Acl'*P+P*Acl+Ccl'*Y*Ccl;
L12 = P*Bcl+Ccl'*Y*Dcl;
L22 = Dcl'*Y*Dcl - Γ*Y; 
@SDconstraint(optModel, hvcat((2,2), L11, L12, L12', L22) <= -myeps*eye(n1+m1,n1+m1));

In [30]:
status = solve(optModel);

----------------------------------------------------------------------------
	SCS v1.2.6 - Splitting Conic Solver
	(c) Brendan O'Donoghue, Stanford University, 2012-2016
----------------------------------------------------------------------------
Lin-sys: sparse-direct, nnz in A = 39
eps = 1.00e-04, alpha = 1.80, max_iters = 20000, normalize = 1, scale = 5.00
Variables n = 6, constraints m = 22
Cones:	sd vars: 22, sd blks: 5
Setup time: 7.33e-04s
----------------------------------------------------------------------------
 Iter | pri res | dua res | rel gap | pri obj | dua obj | kap/tau | time (s)
----------------------------------------------------------------------------
     0|      inf       inf       nan       nan       inf       inf  5.80e-04 
   100|      inf       inf       nan       nan       inf       inf  8.99e-04 
   200|      inf       inf       nan       nan       inf       inf  1.19e-03 
   300|      inf       inf       nan       nan       inf       inf  1.47e-03 
   400

In [31]:
status

:Infeasible